In [1]:
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.callbacks import get_openai_callback

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# explicitly set the API key for embedding
import openai
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']

openai_api_base = os.environ['OPENAI_API_BASE']
azure_development_name = os.environ['AZURE_DEVELOPMENT_NAME']
openai_api_key = os.environ['OPENAI_API_KEY']

In [3]:
"""import pdfminer"""
from langchain.document_loaders import ReadTheDocsLoader, PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS


loader = PyMuPDFLoader("upload/Resume_Jesse_Chow.pdf")
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
db = FAISS.from_documents(documents, embeddings)
resume_retriever = db.as_retriever()

In [4]:
# create job duties 
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

jd = """

Job duties of this Role:

At the core of Accenture's Technology business, you are eager to learn and use that curiosity to solve technology problems through developing, designing and maintaining software products or systems that enable client strategies to improve the way our clients and the world works.

You will be using your versatility and experience to create and support technology solutions that meet client requirements from analysis to implementation. The latest SDLC best practices will be applied to continuously improve the quality and efficiency of the Accenture development teams.

Come JOIN US if you have:
• Hands-on experience in application development and design on web applications that integrate to large scale/ mission critical systems using different technologies
• Strong in Web (Responsive UI, Angular, React, JavaScript, HTML, CSS) Development
• Knowledge on Redux and Saga is a plus
• Interested to explore other technologies integration
• Passionate and keen to develop your profession in technical delivery, and strive to deliver the best design, codes and practice
• Excellent communication skill and the ability to interact professionally with diverse group of stakeholders, internally and externally
• University Degree in Computer Science/Engineering, Information Technology/System, or other relevant disciplines desirable
• High proficiency in both verbal and written English and Cantonese (Mandarin is an added advantage)
• Dynamic and adaptive to the global collaborative project team environment
• Candidates with more related experience can be considered as senior position"""


jd_doc = Document(page_content=jd)
split_doc = text_splitter.split_documents([jd_doc])
job_db = FAISS.from_documents(split_doc, embeddings)
job_retriever = job_db.as_retriever()

In [5]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [6]:
from langchain.chains import RetrievalQA

# setup model in Azure Openai, and the model used to extract information from vectorstore



# model using to extract information from vectorstore
llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version="2023-03-15-preview",
    deployment_name=azure_development_name,
    openai_api_key=openai_api_key,
    openai_api_type='azure',
    temperature=0,
    max_tokens=256
)

resume_prompt_template = """Use the following pieces of context to provide information from the resume of the candidate to help rewrite the question to better assess the candidate.

{context}

Question: {question}
Answer:"""

job_prompt_template = """Use the following pieces of context to provide information of the job roles and duties to help rewrite the question to assess whether the candidate meets requirements included in the context.

{context}

Question: {question}
Answer:"""


RESUME_PROMPT = PromptTemplate(
    template=resume_prompt_template, input_variables=["context", "question"]
)

JOB_PROMPT = PromptTemplate(
    template=job_prompt_template, input_variables=["context", "question"]
)

resume_chain_type_kwargs = {"prompt": RESUME_PROMPT}
job_chain_type_kwargs = {"prompt": JOB_PROMPT}

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=resume_retriever,
    chain_type_kwargs=resume_chain_type_kwargs
)

jd_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=job_retriever,
    chain_type_kwargs=job_chain_type_kwargs
)

In [7]:
"""create tool"""
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

tool_desc = """Use this tool to create interview question related to skills and experience of the candidate. This tool can also be used for create follow up questions from candidate answers."""

jd_desc = """Use this tool to provide information of the job position which the candidate applied to create interview question. This tool can also be used for create follow up questions from candidate answers."""


tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name="resume"
), Tool(
    func=jd_retriever.run,
    description=jd_desc,
    name="job_duties"
)]

In [8]:
"""FAKE chat history"""

memory = ConversationBufferMemory(memory_key="chat_history")
memory.save_context(
    {'input': "hi"},
    {"output" : "Goog morning, please introduce yourself.",})
memory.save_context(
    {'input': 'I am Jesse Chow, have 100 year experience in software development.'},
    {'output': "That's good to hear. What is your current job title?"}
)

In [9]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, example=False), AIMessage(content='Goog morning, please introduce yourself.', additional_kwargs={}, example=False), HumanMessage(content='I am Jesse Chow, have 100 year experience in software development.', additional_kwargs={}, example=False), AIMessage(content="That's good to hear. What is your current job title?", additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

In [10]:
prefix = """You are an experienced career coach. You are coaching a job seeker who just finished a job interview. Given the following Interview Record, AI as the interview and Human as the candidate. You should 3 actionable items for the candidate to improve in interview in point form. Your suggestions should directly refer to the interview record. You have access to the following tools:"""
suffix = """
Interview Record:
{chat_history}

Begin! Refer to the interview record as you write your suggestions. Quote some of the interview record to support your suggestions. Remember, You are talking to the candidate who is the interviewee. Your response should not include the name of the tool you used. Your response should be in point form.

Questions: {input}

{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools=tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)


model = AzureOpenAI(
    openai_api_base=openai_api_base,
    deployment_name="text-davinci-003",
    openai_api_key=openai_api_key,
    temperature=0.2
)

llm_chain = LLMChain(llm=model, prompt=prompt)

evaluate_agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(agent=evaluate_agent, tools=tools, verbose=True, memory=memory)

In [11]:
agent_chain.run(input="Please give me 3 actionable items to increase my chance to get the job.")



> Entering new AgentExecutor chain...
Thought: I need to think about what the interviewer asked and what the candidate answered
Action: resume
Action Input: Jesse Chow's software development experience
Observation: What programming languages, web technologies, and databases is Jesse Chow proficient in, and what cloud services has he worked with? Can he provide examples of web applications and mobile apps he has developed using Next.js, React, Vue, and React Native, and what role did he play in their development? Additionally, can he describe his experience with backend API development using Nest.js, MongoDB, and MySQL, as well as his system design and DevOps experience with cloud services such as AWS and GCP?
Thought: I now know what the candidate needs to improve in the interview
Final Answer: 
1. Prepare to answer questions about your software development experience in detail.
2. Practice answering questions about the job duties and responsibilities of the position you applied for.

'1. Prepare to answer questions about your software development experience in detail.\n2. Practice answering questions about the job duties and responsibilities of the position you applied for.\n3. Be prepared to provide examples of projects you have worked on and the role you played in their development.'

: 

In [3]:
import redis

rs = redis.Redis("redis://redis.internal.happyhill-5bfa4661.eastus.azurecontainerapps.io:6379")


ModuleNotFoundError: No module named 'redis'